# Gezamenlijke, Marginale en Conditionele Kansen

In dit labo werken we met combinaties van random variabelen. We bestuderen gezamenlijke kansverdelingen, leren marginaliseren en berekenen conditionele kansen.

**Leerdoelen:**
- Gezamenlijke (joint) kansverdelingen implementeren en visualiseren
- Marginale verdelingen afleiden uit joint verdelingen
- Conditionele kansen berekenen en interpreteren
- Het concept van (on)afhankelijkheid begrijpen en toetsen

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats

# Set random seed
rng = np.random.default_rng(42)

## Gezamenlijke Kansverdeling: Weer en Verplaatsingsmodus

Wanneer we de relatie tussen weertype en verplaatsingsmodus bekijken, hebben we te maken met twee random variabelen. De gezamenlijke verdeling beschrijft de kans op elke mogelijke combinatie.

**Scenario:**
- **X (Weertype):** Zonnig (1), Bewolkt (2), Regenachtig (3)
- **Y (Verplaatsingsmodus):** Te voet (1), Auto (2), Tram/Bus (3), Trein (4)

## ✍️
Construeer de joint probability table voor weertype en verplaatsingsmodus en beantwoord enkele kansberekening vragen.

**Probabiliteitswaarden (matrix representatie):**

|  | Te voet (y=1) | Auto (y=2) | Tram/Bus (y=3) | Trein (y=4) |
|---|---|---|---|---|
| **Zonnig (x=1)** | 0.20 | 0.10 | 0.12 | 0.08 |
| **Bewolkt (x=2)** | 0.10 | 0.08 | 0.08 | 0.04 |
| **Regenachtig (x=3)** | 0.03 | 0.07 | 0.07 | 0.03 |


In [2]:
# Create joint probability table for weather and transportation mode
# X: Weather (1=Sunny, 2=Cloudy, 3=Rainy)
# Y: Transportation (1=Walking, 2=Car, 3=Tram/Bus, 4=Train)

# Define the joint probability matrix
# People walk more when sunny, use car/bus more when rainy
joint_prob = np.array(
    [
        [0.20, 0.10, 0.12, 0.08],  # Sunny (P(x=1) = 0.50)
        [0.10, 0.08, 0.08, 0.04],  # Cloudy (P(x=2) = 0.30)
        [0.03, 0.07, 0.07, 0.03],  # Rainy (P(x=3) = 0.20)
    ]
)

weather_labels = ["Zonnig", "Bewolkt", "Regenachtig"]
transport_labels = ["Te voet", "Auto", "Tram/Bus", "Trein"]

# Create DataFrame for better visualization
joint_df = pd.DataFrame(
    joint_prob,
    index=[f"{w} (x={i + 1})" for i, w in enumerate(weather_labels)],
    columns=[f"{t} (y={j + 1})" for j, t in enumerate(transport_labels)],
)

print("Joint Probability Distribution P(x, y):")
print(joint_df)
print(f"\nTotal probability: {joint_prob.sum():.1f}")

Joint Probability Distribution P(x, y):
                   Te voet (y=1)  Auto (y=2)  Tram/Bus (y=3)  Trein (y=4)
Zonnig (x=1)                0.20        0.10            0.12         0.08
Bewolkt (x=2)               0.10        0.08            0.08         0.04
Regenachtig (x=3)           0.03        0.07            0.07         0.03

Total probability: 1.0


## ✍️
Bereken de volgende kansen:
1. $P(x=1, y=1)$ - zonnig weer EN te voet
2. $P(x=3, y \in \{2,3\})$ - regenachtig EN (auto OF tram/bus)
3. $P(y=4)$ - trein (ongeacht het weer)

In [3]:
# 1. P(x=1, y=1) - Sunny and Walking
p_sunny_walking = joint_prob[0, 0]
print(f"1. P(x=1, y=1) = P(Zonnig, Te voet) = {p_sunny_walking:.3f}")

# 2. P(x=3, y∈{2,3}) - Rainy and (Car OR Tram/Bus)
p_rainy_car_or_bus = joint_prob[2, 1] + joint_prob[2, 2]
print(f"2. P(x=3, y∈{{2,3}}) = P(Regenachtig, Auto of Tram/Bus) = {p_rainy_car_or_bus:.3f}")

# 3. P(y=4) - Train (regardless of weather) - this is marginal probability
p_train = joint_prob[:, 3].sum()
print(f"3. P(y=4) = P(Trein) = {p_train:.3f}")

1. P(x=1, y=1) = P(Zonnig, Te voet) = 0.200
2. P(x=3, y∈{2,3}) = P(Regenachtig, Auto of Tram/Bus) = 0.140
3. P(y=4) = P(Trein) = 0.150


## Marginale Kansverdeling

De **marginale verdeling** van één variabele verkrijgen we door te sommeren over alle mogelijke waarden van de andere variabele(n).

Voor discrete variabelen: $P(x) = \sum_y P(x, y)$

## ✍️
Bereken de marginale verdelingen $P(x)$ en $P(y)$ uit de joint distributie.

In [4]:
# Calculate marginal distributions
marginal_x = joint_prob.sum(axis=1)  # sum over y (columns) - weather distribution
marginal_y = joint_prob.sum(axis=0)  # sum over x (rows) - transport distribution

print("Marginal distribution P(x) - Weertype:")
for i, (weather, prob) in enumerate(zip(weather_labels, marginal_x, strict=False), start=1):
    print(f"  P(x={i}) = P({weather}) = {prob:.3f}")

print("\nMarginal distribution P(y) - Verplaatsingsmodus:")
for j, (transport, prob) in enumerate(zip(transport_labels, marginal_y, strict=False), start=1):
    print(f"  P(y={j}) = P({transport}) = {prob:.3f}")

print(f"\nVerification:")
print(f"  Sum of P(x): {marginal_x.sum():.1f}")
print(f"  Sum of P(y): {marginal_y.sum():.1f}")

Marginal distribution P(x) - Weertype:
  P(x=1) = P(Zonnig) = 0.500
  P(x=2) = P(Bewolkt) = 0.300
  P(x=3) = P(Regenachtig) = 0.200

Marginal distribution P(y) - Verplaatsingsmodus:
  P(y=1) = P(Te voet) = 0.330
  P(y=2) = P(Auto) = 0.250
  P(y=3) = P(Tram/Bus) = 0.270
  P(y=4) = P(Trein) = 0.150

Verification:
  Sum of P(x): 1.0
  Sum of P(y): 1.0


## Conditionele Kansverdeling

De **conditionele kans** $P(y|x)$ geeft de kans op $y$ gegeven dat $x$ heeft plaatsgevonden:

$$P(y|x) = \frac{P(x, y)}{P(x)}$$

## ✍️
Bereken de conditionele verdeling $P(y|x \in \{2,3\})$: wat is de kans op elke verplaatsingsmodus, gegeven dat het bewolkt is of regent?

In [5]:
# Conditional distribution P(y | x in {2, 3}) - Transport mode given Cloudy or Rainy weather
x_values = [2, 3]  # Cloudy or Rainy
x_indices = [x - 1 for x in x_values]

# P(y | x in {2, 3}) = P(x in {2, 3}, y) / P(x in {2, 3})
conditional_y_given_cloudy_or_rainy = (
    joint_prob[x_indices, :].sum(axis=0) / marginal_x[x_indices].sum()
)

print(
    f"Conditional distribution P(y | x={x_values}) - Verplaatsingsmodus gegeven Bewolkt of Regenachtig weer:"
)
for j, (transport, prob) in enumerate(
    zip(transport_labels, conditional_y_given_cloudy_or_rainy, strict=False), start=1
):
    print(f"  P(y={j} | x={x_values}) = P({transport} | Bewolkt of Regen) = {prob:.4f}")

print(f"\nVerification: sum = {conditional_y_given_cloudy_or_rainy.sum():.1f}")

# Compare with marginal - they should be different (variables are dependent)
print("\nComparison with marginal P(y):")
print(f"  P(y|x={x_values}) ≈ P(y)? {np.allclose(conditional_y_given_cloudy_or_rainy, marginal_y)}")
print(
    f"  → Variables are {'independent' if np.allclose(conditional_y_given_cloudy_or_rainy, marginal_y) else 'dependent'}!"
)

Conditional distribution P(y | x=[2, 3]) - Verplaatsingsmodus gegeven Bewolkt of Regenachtig weer:
  P(y=1 | x=[2, 3]) = P(Te voet | Bewolkt of Regen) = 0.2600
  P(y=2 | x=[2, 3]) = P(Auto | Bewolkt of Regen) = 0.3000
  P(y=3 | x=[2, 3]) = P(Tram/Bus | Bewolkt of Regen) = 0.3000
  P(y=4 | x=[2, 3]) = P(Trein | Bewolkt of Regen) = 0.1400

Verification: sum = 1.0

Comparison with marginal P(y):
  P(y|x=[2, 3]) ≈ P(y)? False
  → Variables are dependent!
